In [2]:
import pandas as pd
import numpy as np
import scipy.spatial.distance as scidist
import json

In [3]:
from math import radians, degrees, sin, cos, asin, acos, sqrt

In [4]:
def great_circle_dist(lonlat1,lonlat2):
    '''https://medium.com/@petehouston/calculate-distance-of-two-locations-on-earth-using-python-1501b1944d97'''
    lon1,lat1=lonlat1
    lon2,lat2=lonlat2
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    return 6371 * (
        acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lon1 - lon2))
    )

In [5]:
hs = pd.read_csv('OEFFHALTESTOGD.csv')

In [6]:
# LTYP 4 = subway
hs = hs.loc[hs['LTYP'] == 4]

In [7]:
hs.drop(labels=['FID','OBJECTID','HTXT','DIVA_ID','WEBLINK1','SE_ANNO_CAD_DATA','LTYP'], axis=1,inplace=True)

In [8]:
hs.reset_index(inplace=True)

In [9]:
hs["LINES"] =hs['HLINIEN'].str.findall(r'(?P<LINE>U\d{1}[EZ]?)').str.join(',')

In [10]:
hspos = hs.SHAPE.str.extract("POINT \((?P<lon>\d{2}\.\d{10,15}) (?P<lat>\d{2}\.\d{10,15})\)")

In [11]:
hs = pd.concat([hs, hspos],axis=1)

In [12]:
hs.drop(labels=['SHAPE', 'HLINIEN'], axis=1,inplace=True)
hs.reset_index(inplace=True)

In [13]:
hs['lat'] = hs['lat'].astype('float64')

In [14]:
hs['lon'] = hs['lon'].astype('float64')

In [15]:
hs = hs.loc[hs['HTXTK'].isin(['Stephansplatz', 'Schwedenplatz', 'Stubentor', 'Mitte-Landstraße', 'Neubaugasse', 'Ottakring', 'Simmering', 'Oberlaa'])]
hs

,level_0,index,HTXTK,LINES,lon,lat
4,4,4695,Stubentor,U3,16.380276,48.207068
9,9,4700,Neubaugasse,U3,16.350201,48.198291
12,12,5021,Mitte-Landstraße,"U3,U4",16.384588,48.206044
13,13,5025,Simmering,U3,16.420009,48.170159
27,27,5046,Stephansplatz,"U1,U3",16.371635,48.208134
53,53,5221,Oberlaa,U1,16.399554,48.142395
57,57,5230,Ottakring,U3,16.311672,48.212025
92,92,5546,Schwedenplatz,U4,16.377896,48.211750


In [16]:
hs_dist = scidist.squareform(scidist.pdist(hs[['lon','lat']], great_circle_dist))

In [17]:
lines = list(sorted(set([item for sublist in hs['LINES'].str.split(',').tolist() for item in sublist])))
lines_vec = [ list(map(lambda l: 1 if l in lines_at_station else 0, lines)) for lines_at_station in hs['LINES'].str.split(',').tolist()]
hs_dist_lines = scidist.squareform(scidist.pdist(np.array(lines_vec), scidist.jaccard))

In [18]:
with open('wienerlinien_sm.json', 'w', encoding='utf8') as f:
    jsonstr = {
        'E': hs['HTXTK'].tolist(),
        'EA': hs_dist.tolist(),
        'SR': hs['LINES'].str.split(',').tolist(),
        'S': lines,
        'SA': hs_dist_lines.tolist()
    }
    json.dump(jsonstr, f, ensure_ascii=False)